## Luck vs. Skill (Emerging Markets Edition)

Goal of this notebook is to test out the tools I made in the python file `luck_v_skill.py`, as well as try out some new data.

In [1]:
import sys
sys.path.insert(0, '/Users/danielsalas/Projects/Fama-French-Funds/src')

from luck_v_skill import *
%load_ext autoreload
%autoreload 2

In [2]:
# Load data from local files
funds = pd.read_csv('../data/EM Funds.csv',parse_dates=['Dates'],index_col='Dates')
factors = pd.read_csv('../data/EM Factors.csv',parse_dates=['Dates'],index_col='Dates')

funds.shape, factors.shape

((214, 812), (214, 9))

In [3]:
# Start counting how long this program takes
start_time = time.time()

# initialize instance of the AlphaEvaluator with fund and factor data
AEV = AlphaEvaluator(fund_data=funds,factor_data=factors)

# Leave all the factors in for now.
# factors = ['EMRF','HML','SMB','RF']
AEV.fit(min_obs=120)
AEV.simulate(n_simulations=1000,random_seed=2,sim_std=0,sim_cutoff=15,verbose=True)
AEV.get_percentiles()
data_a, data_t = AEV.data_a.copy(), AEV.data_t.copy()

# Rename data columns
data_a.columns = ['Actual','Sim Avg (0.00)', '%<Act (0.00)']
data_t.columns = ['Actual','Sim Avg (0.00)', '%<Act (0.00)']

for stdev in np.arange(1,15)/4:
    # Rerun simulation with new standard deviation injection for alpha
    AEV.simulate(n_simulations=1000,random_seed=2,sim_std=stdev,sim_cutoff=15,verbose=True)
    
    # Update Percentile Data
    AEV.get_percentiles()
    
    # Alphas
    data_a['Sim Avg ({:.2f})'.format(stdev)] = AEV.data_a['Sim Avg']
    data_a['%<Act ({:.2f})'.format(stdev)] = AEV.data_a['%<Act']
    
    # t-statistics for Alpha
    data_t['Sim Avg ({:.2f})'.format(stdev)] = AEV.data_t['Sim Avg']
    data_t['%<Act ({:.2f})'.format(stdev)] = AEV.data_t['%<Act']
    
data_a.to_csv('../EM-luck-vs-skill-alphas.csv')
data_t.to_csv('../EM-luck-vs-skill-tstats.csv')

Annual standard deviation: 0.00, Standard deviation alpha: 0.00
Starting 206,000 regressions...
Simulations complete in 301.31 seconds!
Annual standard deviation: 0.25, Standard deviation alpha: 0.07
Starting 206,000 regressions...
Simulations complete in 298.50 seconds!
Annual standard deviation: 0.50, Standard deviation alpha: 0.14
Starting 206,000 regressions...
Simulations complete in 297.45 seconds!
Annual standard deviation: 0.75, Standard deviation alpha: 0.22
Starting 206,000 regressions...
Simulations complete in 298.31 seconds!
Annual standard deviation: 1.00, Standard deviation alpha: 0.29
Starting 206,000 regressions...
Simulations complete in 298.38 seconds!
Annual standard deviation: 1.25, Standard deviation alpha: 0.36
Starting 206,000 regressions...
Simulations complete in 296.97 seconds!
Annual standard deviation: 1.50, Standard deviation alpha: 0.43
Starting 206,000 regressions...
Simulations complete in 299.19 seconds!
Annual standard deviation: 1.75, Standard deviat

In [4]:
t_diff = time.time()-start_time
minutes = int(np.floor(t_diff/60))
seconds = t_diff - 60*minutes
print("{} minutes and {:.2f} seconds elapsed for this program".format(minutes,seconds))

75 minutes and 9.83 seconds elapsed for this program


In the above cells, I left my computer and it went to sleep, which paused the machine. Should only take about 65 to 75 minutes total to run. 